In [29]:
import pickle
import numpy as np
import pandas as pd
from implicit.bpr import BayesianPersonalizedRanking
from rectools.dataset import Interactions, Dataset
from rectools.model_selection import TimeRangeSplitter
from rectools.metrics import MAP, MeanInvUserFreq, calc_metrics, MRR, NDCG

from implicit.als import AlternatingLeastSquares

from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel, PopularModel
from tqdm import tqdm

from lightfm import LightFM

In [30]:
interactions_df = pd.read_csv('../artifacts/data_original/interactions.csv')
users = pd.read_csv('../artifacts/data_original/users.csv')
items = pd.read_csv('../artifacts/data_original/items.csv')

interactions_df.rename(columns={'last_watch_dt': Columns.Datetime,
                                'total_dur': Columns.Weight}, inplace=True) 

# Мы хотим учитывать юзеров с 1 или двумя просмотрами, так как этого недостаточно чтобы говорить о пользователях хоть что-нибудь
user_ids_all = interactions_df.groupby(Columns.User)[Columns.Item].nunique().reset_index(name='unique_items_count')
hot_users = user_ids_all[user_ids_all['unique_items_count'] > 20][Columns.User]
interactions_df_hot_users = interactions_df[interactions_df[Columns.User].isin(hot_users)]


users = users[users[Columns.User].isin(interactions_df_hot_users[Columns.User])]
interactions_df_hot_users = interactions_df_hot_users[interactions_df_hot_users[Columns.User].isin(users[Columns.User])]
items = items[items[Columns.Item].isin(interactions_df_hot_users[Columns.Item])]

interactions = Interactions(interactions_df_hot_users)   
catalog = interactions.df[Columns.Item].unique()

interactions.df

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
6,1016458,354,2021-08-14,1672.0,25.0
14,5324,8437,2021-04-18,6598.0,92.0
18,927973,9617,2021-06-19,8422.0,100.0
...,...,...,...,...,...
5476213,706423,3384,2021-03-13,5937.0,100.0
5476223,489587,6945,2021-05-18,229.0,4.0
5476226,435089,13475,2021-07-06,4624.0,85.0
5476239,610017,7107,2021-05-10,1133.0,75.0


In [31]:
interactions_df_hot_users

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
6,1016458,354,2021-08-14,1672.0,25.0
14,5324,8437,2021-04-18,6598.0,92.0
18,927973,9617,2021-06-19,8422.0,100.0
...,...,...,...,...,...
5476213,706423,3384,2021-03-13,5937.0,100.0
5476223,489587,6945,2021-05-18,229.0,4.0
5476226,435089,13475,2021-07-06,4624.0,85.0
5476239,610017,7107,2021-05-10,1133.0,75.0


In [32]:
items

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15958,6443,series,Полярный круг,Arctic Circle,2018.0,"драмы, триллеры, криминал","Финляндия, Германия",NaN,16.0,NaN,Ханну Салонен,"Иина Куустонен, Максимилиан Брюкнер, Пихла Вии...","Во время погони за браконьерами по лесу, сотру...","убийство, вирус, расследование преступления, н..."
15959,2367,series,Надежда,NaN,2020.0,"драмы, боевики",Россия,0.0,18.0,NaN,Елена Хазанова,"Виктория Исакова, Александр Кузьмин, Алексей М...",Оригинальный киносериал от создателей «Бывших»...,"Надежда, 2020, Россия"
15960,10632,series,Сговор,Hassel,2017.0,"драмы, триллеры, криминал",Россия,0.0,18.0,NaN,"Эшреф Рейбрук, Амир Камдин, Эрик Эгер","Ола Рапас, Алиетт Офейм, Уильма Лиден, Шанти Р...",Криминальная драма по мотивам романов о шведск...,"Сговор, 2017, Россия"
15961,4538,series,Среди камней,Darklands,2019.0,"драмы, спорт, криминал",Россия,0.0,18.0,NaN,"Марк О’Коннор, Конор МакМахон","Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родд...",Семнадцатилетний Дэмиен мечтает вырваться за п...,"Среди, камней, 2019, Россия"


In [33]:
N_SPLITS = 1
TEST_SIZE = '14D'

cv = TimeRangeSplitter(
    test_size=TEST_SIZE,
    n_splits=N_SPLITS,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)
cv.get_test_fold_borders(interactions)

[(Timestamp('2021-08-09 00:00:00', freq='14D'),
  Timestamp('2021-08-23 00:00:00', freq='14D'))]

In [34]:
K_RECOS = 10
RANDOM_STATE = 42
NUM_THREADS = 8
N_FACTORS = (4,)
N_EPOCHS = 1 # Lightfm
LEARNING_RATE = 0.001 # Lightfm

metrics = {
    "precision@10": Precision(k=10),
    "accuracy@10": Recall(k=10),
    "map@10": MAP(k=10),
    "mrr@10": MRR(k=10),
    "ndcg@10": NDCG(k=10),
    "precision@5": Precision(k=5),
    "accuracy@5": Recall(k=5),
    "map@5": MAP(k=5),
    "mrr@5": MRR(k=5),
    "ndcg@5": NDCG(k=5),
    "precision@1": Precision(k=1),
    "accuracy@1": Recall(k=1),
    "map@1": MAP(k=1),
    "mrr@1": MRR(k=1),
    "ndcg@1": NDCG(k=1),
}

models = {
    # "popular": PopularModel(),
}

implicit_models = {
    'ALS': AlternatingLeastSquares,
    # 'BPR': BayesianPersonalizedRanking,
}
for implicit_name, implicit_model in implicit_models.items():
    for n_factors in N_FACTORS:
        models[f"{implicit_name}_{n_factors}"] = ImplicitALSWrapperModel(
            model=implicit_model(
                factors=n_factors, 
                random_state=RANDOM_STATE, 
                num_threads=NUM_THREADS
            )
        )

lightfm_losses = ('logistic', 'bpr', 'warp')

for loss in lightfm_losses:
    for n_factors in N_FACTORS:
        models[f"LightFM_{loss}_{n_factors}"] = LightFMWrapperModel(
            LightFM(
                no_components=n_factors, 
                loss=loss, 
                random_state=RANDOM_STATE,
            ),
            epochs=10,
            num_threads=NUM_THREADS,
        )

In [35]:
user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
24,269408,Ж,sex
27,384532,М,sex
66,216495,М,sex
81,515668,М,sex
136,824452,М,sex


In [36]:
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"

item_features = pd.concat((genre_feature, content_feature))

In [37]:
%%time
    
cv.get_test_fold_borders(interactions)

results = []

fold_iterator = cv.split(interactions, collect_fold_stats=True)

for train_ids, test_ids, fold_info in tqdm((fold_iterator), total=cv.n_splits):
    df_train = interactions.df.iloc[train_ids]
    df_test = interactions.df.iloc[test_ids][Columns.UserItem]
    test_users = np.unique(df_test[Columns.User])
    
    catalog = df_train[Columns.Item].unique()
    
    user_features_train = user_features[user_features["id"].isin(df_train[Columns.User])]
    item_features_train = item_features[item_features["id"].isin(df_train[Columns.Item])]    

    dataset = Dataset.construct(
        interactions_df=df_train,
        # user_features_df=user_features_train,
        # cat_user_features=["sex", "age", "income"],
        # item_features_df=item_features_train,
        # cat_item_features=["genre", "content_type"],
    )


    for model_name, model in models.items():
        print(f"Fitting model {model_name}...")
        model_quality = {'model': model_name}
    
        model.fit(dataset)
        recos = model.recommend(
            users=test_users,
            dataset=dataset,
            k=K_RECOS,
            filter_viewed=True,
        )
        metric_values = calc_metrics(
                metrics,
                reco=recos,
                interactions=df_test,
                prev_interactions=interactions.df,
                catalog=catalog,
            )
        model_quality.update(metric_values)
        results.append(model_quality)
results

  0%|          | 0/1 [00:00<?, ?it/s]

Fitting model popular...
Fitting model ALS_4...
Fitting model LightFM_logistic_4...


  0%|          | 0/1 [00:13<?, ?it/s]


ValueError: Not all estimated parameters are finite, your model may have diverged. Try decreasing the learning rate or normalising feature values and sample weights

In [38]:
df_quality = pd.DataFrame(results).T
df_quality.columns = df_quality.iloc[0]
df_quality.drop('model', inplace=True)

In [39]:
df_quality.style.highlight_max(color='lightgreen', axis=1).data

model,popular,ALS_4
precision@10,0.039198,0.029481
accuracy@10,0.075828,0.057439
precision@5,0.060346,0.031825
accuracy@5,0.060579,0.030115
precision@1,0.103248,0.014853
accuracy@1,0.021566,0.002445
ndcg@10,0.051512,0.02767
ndcg@5,0.069512,0.027854
ndcg@1,0.103248,0.014853
mrr@10,0.159545,0.061522


In [40]:
user_features

,id,value,feature
24,269408,Ж,sex
27,384532,М,sex
66,216495,М,sex
81,515668,М,sex
136,824452,М,sex
...,...,...,...
840084,1029836,income_40_60,income
840098,605991,income_20_40,income
840121,476507,income_40_60,income
840137,486585,income_20_40,income


In [41]:
item_features

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre
...,...,...,...
15958,6443,series,content_type
15959,2367,series,content_type
15960,10632,series,content_type
15961,4538,series,content_type


In [42]:
df_train

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
14,5324,8437,2021-04-18,6598.0,92.0
18,927973,9617,2021-06-19,8422.0,100.0
19,215229,7793,2021-08-01,873.0,14.0
...,...,...,...,...,...
5476213,706423,3384,2021-03-13,5937.0,100.0
5476223,489587,6945,2021-05-18,229.0,4.0
5476226,435089,13475,2021-07-06,4624.0,85.0
5476239,610017,7107,2021-05-10,1133.0,75.0
